<a href="https://colab.research.google.com/github/nomomon/Amazon_Product_Description_Texta.ai/blob/main/User_Anime_Ranting_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#@title Kaggle API
username = str(input("username"))
key = str(input("key"))

f = open("kaggle.json", "w")
f.write('{"username":"'+username+'","key":"'+key+''}')
f.close()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

import os, re
from zipfile import ZipFile

with ZipFile('/content/anime-recommendation-database-2020.zip', 'r') as zipObj:
   zipObj.extractall()

def purge(dir, pattern):
    for f in os.listdir(dir):
        if pattern in f:
            os.remove(os.path.join(dir, f))

purge("/content/", ".zip")
purge("/content/", ".json")

 99% 657M/661M [00:07<00:00, 96.2MB/s]
100% 661M/661M [00:07<00:00, 88.5MB/s]


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Data Preprocessing

In [6]:
anime = pd.read_csv("/content/anime.csv")
ratings = pd.read_csv("/content/animelist.csv")

In [7]:
ratings.head(5)

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


In [8]:
ratings["rating"].value_counts()

0     46827035
8     15422150
7     14244633
9     10235934
6      7543377
10     7144392
5      4029645
4      1845854
3       905700
2       545339
1       480688
Name: rating, dtype: int64

In the dataset, if a person hasn't left a rating to an anime, it was marked as a zero. Let's remove the zeros, as they will just introduce noise later on. Might as well remove the `watching_status` and `watched_episodes`. They are usefull pieces of data, but we won't use them in this tutorial.

In [9]:
ratings = ratings[ratings["rating"] > 0]
ratings = ratings.drop(columns = ["watching_status", "watched_episodes"])

In [10]:
num_users = ratings["user_id"].nunique()
num_anime = ratings["anime_id"].nunique()

print(f"There are a total of {num_users} users found.")
print(f"There are a total of {num_anime} anime found.")

print("Max user id:", ratings["user_id"].max())
print("Max anime id:", ratings["anime_id"].max())

There are a total of 313670 users found.
There are a total of 17172 anime found.
Max user id: 353404
Max anime id: 48456


If we take a look at the anime and user ids, we can notice that some values there are missing. Let's make a two helper functions, for easy conversion between dataset ids and embedding ids.

In [11]:
embId2user = sorted(ratings["user_id"].unique())
embId2anime = sorted(ratings["anime_id"].unique())

In [12]:
user2embId = {v: k for k, v in enumerate(embId2user)}
anime2embId = {v: k for k, v in enumerate(embId2anime)}

Now, let's make the dataset. It will look like a tuples of `(user_id, anime_embId, rating)`. We'll make 3 sets: `train`, `test` and `val`.

In [13]:
np.random.seed(seed = 42)

def makeDataSet(df, split=0.95):
    n = df.to_numpy()
    
    n = np.random.permutation(n)

    x = n[:, :2]
    y = n[:, 2]

    for i in range(x.shape[0]):
        x[i][0] = user2embId[x[i][0]]
        x[i][1] = anime2embId[x[i][1]]

    s1 = int(split * n.shape[0])
    s2 = s1 + int((1 - split) * n.shape[0] / 10)
    
    return (x[:s1], y[:s1], x[s1:s2], y[s1:s2], x[s2:], y[s2:])

In [14]:
x_train, y_train, x_test, y_test, x_val, y_val = makeDataSet(ratings)

# Models

In [15]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Matrix Factorization

In [20]:
class MatrixFactorizationModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim):
        super(MatrixFactorizationModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embeddings = tf.keras.layers.Embedding(num_items, embedding_dim)

        self.user_biases = tf.keras.layers.Embedding(num_users, 1)
        self.item_biases = tf.keras.layers.Embedding(num_items, 1)

        self.bias = tf.Variable(tf.zeros([1]))

        self.dropout = tf.keras.layers.Dropout(.5)

    def call(self, inputs, training = False):
        user_ids = inputs[:, 0]
        item_ids = inputs[:, 1]

        user_embedding = self.user_embeddings(user_ids) + self.user_biases(user_ids)
        item_embedding = self.item_embeddings(item_ids) + self.item_biases(item_ids)

        if training:
            user_embedding = self.dropout(user_embedding, training = training)
            item_embedding = self.dropout(item_embedding, training = training)

        user_embedding = tf.reshape(user_embedding, [-1, self.embedding_dim])
        item_embedding = tf.reshape(item_embedding, [-1, self.embedding_dim])

        dot = tf.keras.layers.Dot(axes=1)([user_embedding, item_embedding]) + self.bias

        return dot

In [21]:
mf_model = MatrixFactorizationModel(num_users = num_users, 
                                    num_items = num_anime, 
                                    embedding_dim = 64)

mf_model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.MeanSquaredError(),
    metrics = [
        tf.keras.metrics.RootMeanSquaredError("RMSE")
    ],
    run_eagerly = True
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

history = mf_model.fit(x = x_train, y = y_train, 
                    batch_size = 64, 
                    epochs = 100, 
                    steps_per_epoch = 1000,
                    callbacks = [callback],
                    # validation_data = (x_test, y_test),
                    # validation_steps = 10,
                    # validation_batch_size = 64
                    )

Epoch 1/100
1000/1000 [==============================] - 34s 31ms/step - loss: 3.1257 - RMSE: 1.7680
Epoch 2/100
1000/1000 [==============================] - 30s 30ms/step - loss: 3.1056 - RMSE: 1.7623
Epoch 3/100
1000/1000 [==============================] - 30s 30ms/step - loss: 3.1381 - RMSE: 1.7715
Epoch 4/100
1000/1000 [==============================] - 30s 30ms/step - loss: 3.0802 - RMSE: 1.7551
Epoch 5/100
1000/1000 [==============================] - 30s 30ms/step - loss: 3.1569 - RMSE: 1.7768
Epoch 6/100
1000/1000 [==============================] - 30s 30ms/step - loss: 3.1054 - RMSE: 1.7622
Epoch 7/100
1000/1000 [==============================] - 30s 30ms/step - loss: 3.1338 - RMSE: 1.7703


In [ ]:
# mf_model.save_weights(
#     "/content/drive/MyDrive/AnimeRecommendations/MatrixFactorizationModel/model", overwrite=True
# )

In [22]:
mf_model.load_weights("/content/drive/MyDrive/AnimeRecommendations/MatrixFactorizationModel/model")

## Neural Network

In [16]:
class NeuralNetworkModel(tf.keras.Model):
    def __init__(self, num_users, num_items, embedding_dim):
        super(NeuralNetworkModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = tf.keras.layers.Embedding(num_users, embedding_dim)
        self.item_embeddings = tf.keras.layers.Embedding(num_items, embedding_dim)

        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='relu')

        self.concat = tf.keras.layers.Concatenate()
        self.dropout = tf.keras.layers.Dropout(.5)

    def call(self, inputs, training = False):
        user_ids = inputs[:, 0]
        item_ids = inputs[:, 1]

        user_embedding = self.user_embeddings(user_ids)
        item_embedding = self.item_embeddings(item_ids)

        if training:
            user_embedding = self.dropout(user_embedding, training = training)
            item_embedding = self.dropout(item_embedding, training = training)

        user_embedding = tf.reshape(user_embedding, [-1, self.embedding_dim])
        item_embedding = tf.reshape(item_embedding, [-1, self.embedding_dim])

        x = self.concat([user_embedding, item_embedding])
        x = self.dense1(x)
        x = self.dense2(x)

        return x

In [17]:
nn_model = NeuralNetworkModel(num_users = num_users, 
                              num_items = num_anime, 
                              embedding_dim = 64)

nn_model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.MeanSquaredError(),
    metrics = [
        tf.keras.metrics.RootMeanSquaredError("RMSE")
    ],
    run_eagerly = True
)

In [47]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

history = nn_model.fit(x = x_train, y = y_train, 
                       batch_size = 64, 
                       epochs = 100, 
                       steps_per_epoch = 1000,
                       callbacks = [callback],
                     # validation_data = (x_test, y_test),
                     # validation_steps = 10,
                     # validation_batch_size = 64
                    )

Epoch 1/100
1000/1000 [==============================] - 29s 26ms/step - loss: 9.8189 - RMSE: 3.1335
Epoch 2/100
1000/1000 [==============================] - 26s 26ms/step - loss: 2.6797 - RMSE: 1.6370
Epoch 3/100
1000/1000 [==============================] - 26s 26ms/step - loss: 2.5304 - RMSE: 1.5907
Epoch 4/100
1000/1000 [==============================] - 26s 26ms/step - loss: 2.4396 - RMSE: 1.5619
Epoch 5/100
1000/1000 [==============================] - 26s 26ms/step - loss: 2.3850 - RMSE: 1.5443
Epoch 6/100
1000/1000 [==============================] - 26s 26ms/step - loss: 2.3514 - RMSE: 1.5334
Epoch 7/100
1000/1000 [==============================] - 26s 26ms/step - loss: 2.3309 - RMSE: 1.5267
Epoch 8/100
1000/1000 [==============================] - 26s 26ms/step - loss: 2.2873 - RMSE: 1.5124
Epoch 9/100
1000/1000 [==============================] - 26s 26ms/step - loss: 2.2818 - RMSE: 1.5106
Epoch 10/100
1000/1000 [==============================] - 26s 26ms/step - loss: 2.2580 - RM

In [48]:
# nn_model.save_weights(
#     "/content/drive/MyDrive/AnimeRecommendations/NeuralNetworkModel/model", overwrite=True
# )

In [18]:
nn_model.load_weights("/content/drive/MyDrive/AnimeRecommendations/NeuralNetworkModel/model")

## Comparing the models

In [23]:
mf_model.evaluate(x = x_val, y = y_val)

87747/87747 [==============================] - 629s 7ms/step - loss: 3.0599 - RMSE: 1.7492


[3.059866428375244, 1.749247431755066]

In [19]:
nn_model.evaluate(x = x_val, y = y_val)

87747/87747 [==============================] - 573s 6ms/step - loss: 1.9272 - RMSE: 1.3882


[1.9271608591079712, 1.3882222175598145]